#Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, confusion_matrix

#Data pre-processing

In [ ]:
# Define the preprocessing function for MRI images
def preprocess_image(image):
    image = tf.image.resize(image, [224, 224])  # Resize images to 224x224
    image = tf.image.grayscale_to_rgb(image)  # Convert grayscale to RGB
    return image / 255.0

#Setup datasets

In [ ]:
# Data augmentation and preparation using ImageDataGenerator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

# Setup data generators
train_generator = train_datagen.flow_from_directory(
    'path_to_train_dir',  # Update with training directory path
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    'path_to_validation_dir',  # Update with validation directory path
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

#Train Model

In [ ]:
# Load ResNet50 with pre-trained ImageNet weights, exclude top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Adding custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Final model setup
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

#Evaluation

In [ ]:
# Evaluate model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


# Predictions
predictions = model.predict(test_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)

# True labels
true_classes = test_generator.classes

# Confusion Matrix and Classification Report
cm = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:\n", cm)

report = classification_report(true_classes, predicted_classes, target_names=['Negative', 'Positive'])
print("Classification Report:\n", report)
